# More `numpy`

A more complete discussion of the basic use of `numpy` is given [here](http://www.scipy-lectures.org/intro/numpy/operations.html). These are all things that, based on my experience, you will end up using all the time. This can still only cover the most basic operations -- there is a huge amount of stuff in `numpy`.

- Differences between copies of arrays and references to arrays
- Sorting arrays
- Concatenating, stacking and splitting arrays
- Creating arrays with `repeat`, `reshape`
- Record arrays (recarrays)
- Logical masks and crazy floating point values (`nan` and `inf`)
- `numpy.where` with 2d arrays
- Finding unique values
- Testing if values from one array are in another array, using in1d
- Finding locations of specific values in arrays

## Copies vs. references

In a previous notebook we introduced the distinction between copies of variables and references, and saw how the `copy` module can be used to make copies. Numpy arrays have their own in-built `copy` function which does the same thing, and is the best way to copy them.

In [ ]:
import numpy as np

In [ ]:
x = np.zeros(3,dtype=np.float32)
y = x
z = x.copy()
x[2] = 100.0 # set an element of x

print('x = {}'.format(x))
print('y = {}'.format(y))
print('z = {}'.format(z))
print('y is x? {}'.format(y is x))
print('z is x? {}'.format(z is x))

The point is that `y=x` makes `y` a reference to `x`, rather than a copy, so updating `y` also updates `x`. In practice it can get quite hard to guess whether any of the hundreds of functions in `numpy` will return a copy or a reference to the array you give it as input (in some cases this even depends on the input!). Again, reading the documentation when you're using a new function is important.

However, the bottom line is, when you know really want a copy, explicitly make a copy.

## Sorting arrays

There are two simple ways to sort an array `x`: either `np.sort(x)`, which returns a copy of `x` in sorted order, or `x.sort()`, which sorts x 'in place'. An in-place sort actually moves the values in `x` itself into sorted order.

This choice between `np.something(x)` (copy) and `x.something()` (in place change to `x`) is similar for a lot of `numpy` functions.

In [ ]:
np.random.seed(42)
x = np.random.randint(0,100,size=10) # Some random integers
print('Original x: {}'.format(x))
print('np.sort(x): {}'.format(np.sort(x))) # np.sort returns a copy
print('         x: {}'.format(x))
print('  x.sort(): {}'.format(x.sort()))   # x.sort() returns None...
print('         x: {}'.format(x))          # but now x is sorted!

The next example shows the `argsort` function, which returns a list of `indices` in sorted order, rather than values.

In [ ]:
np.random.seed(42)
x = np.random.randint(0,100,size=10) # Some random integers
a = np.argsort(x)
print(a)          # The *indices* rather than the *values* in sorted order
print(x[a])       # Use the indices to sort the array
print(x[a[::-1]]) # Sort backwards by reversing the indices

This is useful because `numpy` allows you to use this array of indices to index other arrays. So you can now use the indices `a` to put any other array in the same order, not just `x`.

## Concatenating, stacking and splitting

These are all ways to combine multiple arrays into a single array. `vstack` and `hstack` add rows and columns respectively, increasing the dimension of the array (i.e. making two 1d arrays into one 2d array). `vstack` means 'vertical stack' (add more rows) and `hstack` means horizontal stack (add more columns) -- experiments are the easiest way to understand what the difference is. 

`concatenate` joins arrays end-to-end along a particular dimension (by default, the first one). For example, it's the standard way to combine a list of 1d arrays into one big 1d array.

In [ ]:
x = np.arange(10,25)
y = np.arange(30,45)

print('x = {}'.format(x))
print('y = {}'.format(y))
print('concatenated: {}'.format(np.concatenate((x,y))))

v = np.vstack((x,y))
print('shape after vstack: {}'.format(v.shape))

h = np.hstack((x,y))
print('shape after hstack: {}'.format(h.shape))

# In this case, stacking horizontally is the same as concatenating 
print(np.all(h == np.concatenate([x,y])))

There are also very similar functions to split up arrays into smaller arrays. Following on from the previous cell:

In [ ]:
a,b = np.vsplit(v,2)
np.all(a==x) and np.all(b==y) # We should be back to where we started!

This last cell use the `all` function to test if all the elements in a logical array are true.

## Creating arrays, repeat, reshape

There are [loads of different functions](https://docs.scipy.org/doc/numpy/reference/routines.array-creation.html) for creating arrays in `numpy`. These include `zeros` to make an empty array and `repeat` to make an array where every element has a particular value. We saw these in the first `numpy` notebook.

Let's examine how long it takes to create a big array of -1's, using IPython's `%timeit` magic function.

In [ ]:
%timeit np.zeros(1000) - 1

In [ ]:
%timeit np.repeat(-1,1000)

So in this case, the first way is almost 3x faster (on my machine -- you might get a different result). This is helpful to know if you're making very big arrays. If you can think of two ways to do something, then either is fine, unless one of them **really** slows down your program. 

Sometimes we want to make an array with a repeating pattern, like `[1,0,-1,1,0,-1,1,0,0,-1]`. Let's try to do that with repeat:

In [ ]:
np.repeat((1,0,-1),10)

Clearly that's not the right way to do it. The correct function in this case is `tile`:

In [ ]:
np.tile((1,0,-1),10)

The `reshape` function can be used if we want a 2d version

In [ ]:
np.reshape(np.tile((1,0,-1),10), (10,3))

## Record arrays (recarrays)

You can define your own `numpy` data types that are combinations of the basic types, with each element of the new composite type associated with a name. This is useful for storing **structured data**, and the most general way to do that with `numpy` (although in many cases you might find the Table class from `astropy` easier to work with).

Here we define a structure array called `line_data`, with five values. 

In [ ]:
# Set up some data
# (data from Hogg, Bovy & Lang 2010)
line_data = np.array([
       (1, 201.0, 592.0, 9.0, 61.0, -0.84),
       (2, 244.0, 401.0, 4.0, 25.0, 0.31),
       (3, 47.0, 583.0, 11.0, 38.0, 0.64),
       (4, 287.0, 402.0, 7.0, 15.0, -0.27),
       (5, 203.0, 495.0, 5.0, 21.0, -0.33)],
dtype=[('ID', '<i4'), ('x', '<f8'), ('y', '<f8'), ('sigma_x', '<f8'), ('sigma_y', '<f8'), ('rho_xy', '<f8')])

In [ ]:
line_data[0:3] # Show the first three values 

You can see each value has a complicated dtype, made up of 6 elements:

In [ ]:
line_data.dtype

The `line_data` array is **not** a 20x6 array, it's a 20x1 array of elements of this dtype.


In [ ]:
line_data.shape

If you want to know the total number of bytes in each element, you can:

In [ ]:
line_data.dtype.itemsize

**The important thing** is that individual fields can be accessed by name:

In [ ]:
line_data['ID']

To make a record array like this from scratch, use the function `numpy.dtype`, passing a ***list of tuples*** of the form (`name_of_field`, `data_type_of_field`). The data type can either be an existing `dtype`, like `numpy.int32`, or a string. `numpy` will convert the string into a dtype, if it can. The '<' and '>' indicated 'little endian' or 'big endian' representations. For example


In [ ]:
my_dtype = np.dtype([('ID', '|S16'), ('ra', '<f8'), ('dec', '<f8')])
np.zeros(4,dtype=my_dtype) # Make an empty array with four of these elements

The `|S16` is the way to specify a string of fixed length 16 x 2-byte characters.

## Logical masks and crazy floating point values (`nan` and `inf`)

We can make an array of booleans that indicate whether each element in the array meets some criterion we define. These are often called 'masks'.

In [ ]:
mask = (line_data['x'] > 200.0) & (line_data['y'] > 500.0)
mask

We can **filter** the array by using the mask as an index.

In [ ]:
line_data[mask]

As we saw in the first numpy notebook, we can use `numpy.where` or `np.flatnonzero` to find the actual indices of the elements for which the mask is `True`:

In [ ]:
w = np.where(mask)[0] # remember where() returns a tuple, not just an array
print(w)
line_data[w]

Very often you'll end up doing something like taking the `log10` of an array with some negative numbers or dividing an array by another array with zeros in it. In those cases, the special values `nan` (not a number), `inf` (infinity) and `-inf` (negative infinity) will show up in the results, mixed in with the elements where whatever operation you tried *was* mathematically valid. For example:

In [ ]:
crazy_values = np.log10(np.tile((1,0,-1),10)) # This will cause some warnings to appear!
print(crazy_values[0:3])

`numpy` has some routines to make masks of the elements that contain these special values.

In [ ]:
np.where(np.isinf(crazy_values))

In [ ]:
np.where(np.isnan(crazy_values))

The `isfinite` routine returns True for the elements in its argument that are *not* any of these special values.

In [ ]:
np.where(np.isfinite(crazy_values))

___
Further reading:

For advanced/frequent work with masked arrays, there is also the `numpy.ma` module, which defines a new 'masked array' type. This gets quite complicated to work with and I don't use it very often. It will be more useful if you are doing complicated linear algebra.

##  `numpy.where` with 2d arrays

Last week we saw that (for example) `np.where(x>something)` returns a tuple of length 1 if `x` is a 1d array, so to get the actual elements of x that are `>something` we need to get the first (and only) element of the result: `np.where(x>something)[0]`.

What if `x` is 2d? The following example shows how things work pretty naturally in this case.

In [ ]:
x = np.arange(9).reshape((3,3))
print(x.shape)
w = np.where(x > 4)
print(w) # note this is a tuple of length 2
print(x[w])

## Unique values

To find the unique values in an array, use `numpy.unique`.

In [ ]:
x = np.random.randint(0,10,20) # 20 random integers between 0 and 10
print(x)
print(np.unique(x))

Note that the results are sorted. The `numpy.unique` function has some useful optional arguments:

In [ ]:
u, idx, inv, counts = np.unique(x,return_index=True,return_inverse=True,return_counts=True)

You can check what these do yourself. One useful application is to make a list of lengths and offsets of values in a sorted array. Here's a trivial example.

In [ ]:
x = np.random.randint(0,10,200) # 20 random integers between 0 and 10
a = np.argsort(x)
values, offset, length = np.unique(x[a],return_index=True,return_counts=True)
for u,o,l in zip(values,offset,length):
    print('{} | {}'.format(u, x[a][o:o+l] ))

Now imagine that `x` is a label that groups some other properties: for example, the 'galaxy ID' of particles in a cosmological simulation or the 'source ID' of multiple exposures of a set of targets observed with a spectrograph. The lengths and offset arrays can be used to find the properties of the subsets defined by `x`:

In [ ]:
luminosity = np.random.random(200) # Some property associated with each x from the last cell

print('  | Total Luminosity')
for u,o,l in zip(values,offset,length):
    print('{} | {:7.5f}'.format(u, np.sum(luminosity[a][o:o+l])))


There are even more general ways to do something similar (the `pandas` package has many such functions) and more efficient techniques with `numpy`, to get rid of that `for` loop (too advanced for this tutorial, but if you want more details have a look at the documentation for `numpy.ufunc.reduceat` and google for some examples, e.g. of `add.reduceat`). The above example works well in most simple cases. Remember the array has to be sorted!

## Testing if values from one array are in another array, using in1d

`numpy.in1d` provides a quick way to check if each element of an array is in another array, but it doesn't tell you *where* those elements are in the other array.

In [ ]:
odd_numbers = np.arange(1,100,2)
np.in1d([1,2,3,4,5],odd_numbers)

Efficiently finding the index of each matching element in the other array is the point of the next example.

## Finding locations of specific values in arrays

Matching specific values in very long lists of arbitrary integers is a common task; for example, in working with the particle data from cosmological simulations, you might might have a list of particle IDs in one snapshot of the simulation, and want to find the same particles in another snapshot.

Currently, this operation isn't covered by an existing routine in `numpy` or `scipy` (as far as I know!). The nearest thing is in the `pandas` statistics package, which we'll look at next week. However, it's not *too* hard to write such a routine yourself. This is the basis of one of the challenges.

First, we'll make an array of 1000 unique integer values, `BIG_ARRAY`. Since we want the values to be unique but not sorted for this example, this takes a couple of steps to set up:

In [ ]:
np.random.seed(42)
x         = np.random.randint(0,500,200) # 200 random integers between 0 and 500
BIG_ARRAY = np.unique(x)

# Note this trick for getting a random ordering by sorting random numbers
random_order = np.argsort(np.random.random(len(BIG_ARRAY)))
BIG_ARRAY    = BIG_ARRAY[random_order]

BIG_ARRAY    = BIG_ARRAY + 2000 
print(BIG_ARRAY)

I generated 200 random integers between 0 and 500, find only the unique values, then randomize their order. To make it clear that the numbers can be anything for this example, I added 2000 to each of them. 

Now let's say we wanted to find where the value 2059 was in this array. Obviously we could use `where`:

In [ ]:
np.where(BIG_ARRAY == 2201)[0]

and if we wanted to find more values, we might use a loop:

In [ ]:
# Reset the seed, so we get consistent results each time we try this
np.random.seed(232)
small_array = np.random.randint(2100,2300,10) # some values to look for: not all of them will be found!

# Loop
matches = list()                                    
for s in small_array:
    w = np.where(BIG_ARRAY == s)[0]
    if len(w) == 0:
        # If we didn't find anything, store -1
        matches.append(-1)
    else:
        matches.append(w[0])
    
matches = np.array(matches)
print(matches)

However, if either `small_array` or `BIG_ARRAY` are large, this is a **very slow** way to do it, because `np.where` has to compare each value from `small_array` against *every* value in BIG_ARRAY. In most of these cases, we didn't even find a match.

**Solution:** When we're dealing with large arrays, it's usually more efficient to sort `BIG_ARRAY` and use a [**bisection search**](https://en.wikipedia.org/wiki/Binary_search_algorithm). In `numpy` we can do this with the `searchsorted` function.

In [ ]:
# First sort BIG_ARRAY
sorting_idx = np.argsort(BIG_ARRAY)

In [ ]:
# Now use searchsorted (have a look at the docstring for this routine as well)

# What is the *smallest* index in the sorted BIG_ARRAY where I can insert the value '2101'
# without changing the order of BIG_ARRAY?
print(BIG_ARRAY[sorting_idx].searchsorted(2201,side='left'))

# What is the *largest* index in the sorted BIG_ARRAY where I can insert the value '2101'
# without changing the order of BIG_ARRAY?
print(BIG_ARRAY[sorting_idx].searchsorted(2201,side='right'))

Just to demonstrate that's the case, show the indices around 56-57:

In [ ]:
BIG_ARRAY[sorting_idx][[59,60,61,62]]

Notice that doesn't directly tell us which of the indices (56 or 57) is *equal* to  2189, or even that **either** of them are. However, look what happens when we try to find a value that isn't in `BIG_ARRAY`:

In [ ]:
print(BIG_ARRAY[sorting_idx].searchsorted(2202,side='left'))
print(BIG_ARRAY[sorting_idx].searchsorted(2201,side='right'))

This should be enough information to get started with the 'matching values in huge arrays of integers' challenge.

## End of notebook